# Imputacion basada en Modelos

## Preparacion del entorno

### Librerias

In [1]:
import janitor  # limpiar datos
import matplotlib.pyplot as plt # visualizacion de datos
import missingno    # exploracion de valores faltantes
import nhanes.load  # libreria con los datos a analizar
import numpy as np  # calculos numericos
import pandas as pd # trabajar con dataframe
import scipy.stats  # trabjar con estadisticos
import seaborn as sns   #   visualizacion estadistica
# import session_info
# sklearn contiene los modelos a utilizar
import sklearn.compose
import sklearn.impute
import sklearn.preprocessing
import statsmodels.api as sm
import statsmodels.datasets
import statsmodels.formula.api as smf

from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import BayesianRidge, Ridge
from sklearn.neighbors import KNeighborsRegressor
from statsmodels.graphics.mosaicplot import mosaic

### Configuracion de Graficas

In [2]:
%matplotlib inline

# tamaño de los graficos
sns.set(
    rc={
        'figure.figsize':(8, 6)
    }
)

# estilo de los graficos
sns.set_style('whitegrid')

### Importar funciones personalizadas

In [3]:
%run '../utils2.ipynb'

## Cargar datos preparados

In [4]:
%run tratamiento_v_categoricas.ipynb

12.25 %


## Imputacion Simple

### Procesar datos

In [10]:
# sustituimos los valores faltantes de la variable de interes con dummies

nhanes_model_df = (
    nhanes_transformed_df
    # dos variables categoricas, dos numericas
    .select_columns('sugar_gm', 'cigarettes', 'weight', 'vigorous_work')
    .sort_values('weight') # ordenamos en funcion de weight
    # sustituir missing de cigarettes con dummies
    .transform_column(
        'cigarettes',
        lambda x: x.bfill(),
        elementwise=False
    )
    .transform_column(
        'cigarettes',
        lambda x: x.ffill(),
        elementwise=False
    )
    # matriz de sombra para las variables seleccionadas
    .missing.bind_shadow_matrix2(
        True,
        False,
        suffix='_imp',
        only_missing=False
    )
)

nhanes_model_df

,sugar_gm,cigarettes,weight,vigorous_work,sugar_gm_imp,cigarettes_imp,weight_imp,vigorous_work_imp
SEQN,,,,,,,,
99970.0,22.73,1.0,72.0,NaN,False,False,False,True
97877.0,NaN,1.0,80.0,0.0,True,False,False,False
97730.0,54.71,1.0,82.0,0.0,False,False,False,False
96451.0,NaN,1.0,82.0,0.0,True,False,False,False
99828.0,80.89,0.0,85.0,NaN,False,False,False,True
...,...,...,...,...,...,...,...,...
96329.0,74.94,1.0,9999.0,0.0,False,False,False,False
99299.0,120.55,1.0,9999.0,0.0,False,False,False,False
96452.0,133.93,1.0,9999.0,0.0,False,False,False,False


### Preparacion del Modelo

In [12]:
# utilizamos la regresion lineal como modelo

cigarettes_ols = (
    nhanes_model_df
    .pipe(
        lambda df: smf.ols('cigarettes ~ weight + sugar_gm + vigorous_work', data=df)
    )
    .fit()
)

### Generacion de los valores imputados mediante el modelo

In [ ]:
# obtener las observaciones con valores nulos en cigarettes
